In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
feature_names = iris.feature_names
feature_names = [name.replace(' (cm)', '') for name in feature_names]
feature_names = [name.replace(' ', '_') for name in feature_names]
feature_names


['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [3]:
from fuzzycocopython import FuzzyCocoClassifier

metrics_weight = {
    "sensitivity":        1.0,
    "specificity":        1.0,
    "rmse":               0.5,
    "distanceThreshold":  0.01,
    "nb_vars":            0.35, # Dontcare ??
}

model = FuzzyCocoClassifier(
    metrics_weights = metrics_weight,
    threshold       = 0.5,
    nb_rules        = 5,
    max_generations = 20,
)
model.fit(X_train, y_train, feature_names=feature_names, target_name="species")

TypeError: _FuzzyCocoBase.__init__() got an unexpected keyword argument 'metrics_weights'

In [8]:
model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [9]:
model.score(X_test, y_test)

1.0

In [10]:
model.description_

{'fitness': 0.9363519638847665,
 'fitness_metrics_weights': {'sensitivity': 1.0,
  'specificity': 1.0,
  'accuracy': 0.0,
  'ppv': 0.0,
  'rmse': 0.5,
  'rrse': 0.0,
  'rae': 0.0,
  'mse': 0.0,
  'distanceThreshold': 0.01,
  'distanceMinThreshold': 0.0,
  'nb_vars': 0.35,
  'overLearn': 0.0,
  'true_positives': 0.0,
  'false_positives': 0.0,
  'true_negatives': 0.0,
  'false_negatives': 0.0},
 'fuzzy_system': {'variables': {'input': {'sepal_length': {'sepal_length.1': 5.033333333333333,
     'sepal_length.2': 6.766666666666667},
    'sepal_width': {'sepal_width.1': 3.2329411764705887,
     'sepal_width.2': 4.145882352941177},
    'petal_length': {'petal_length.1': 2.1176470588235294,
     'petal_length.2': 5.91764705882353},
    'petal_width': {'petal_width.1': 1.2670588235294118,
     'petal_width.2': 1.8976470588235292}},
   'output': {'species': {'species.1': 0.07058823529411765,
     'species.2': 1.9607843137254901}}},
  'rules': {'rule1': {'antecedents': {'petal_length': {'petal_l

In [11]:
model.rules_

[IF (petal_length is Low AND_min petal_width is Low), THEN (species is Low)]

In [12]:
model.default_rules_

[ELSE (species is High)]

In [13]:
model.variables_

[Name: sepal_length, values: dict_keys(['Low', 'High']),
 Name: sepal_width, values: dict_keys(['Low', 'High']),
 Name: petal_length, values: dict_keys(['Low', 'High']),
 Name: petal_width, values: dict_keys(['Low', 'High']),
 Name: species, values: dict_keys(['Low', 'High'])]